In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
import wfdb
import copy as cp
import scipy.signal as signal
import pickle
from sklearn import preprocessing
from tqdm import tqdm
import os
import re
import pandas as pd
import csv

In [2]:
record_list = [] # Initialize the array that will hold the list of our records

records = 'mit-bih-dataframes/subject_list.csv' # Get our record list like we did in the initial extraction
with open(records) as rfile:# Load our records into the array
    for record in rfile:
        record = record[0:-1] # The -1 removes the newline ("\n") character from the string
        record_list.append(record)

In [3]:
rr_ints = [] # Initialize the array that will hold all of our subjects' RR-Intervals

for idx, subj in enumerate(record_list): # Iterate through our subject ids
    rr_ints.append(np.genfromtxt('mit-bih-rr-intervals/'+str(subj)+'.csv',delimiter=',')) # Add to master array
       #NOTE = change 'mit-bih-rr-intervals/' to the appropriate location

In [4]:
outlier_list=[]
rrs=[]
for idx, subj in tqdm(enumerate(rr_ints)):
    outlier = [[],[]] # I want to store the index and the outlier
    for idx2, rr in enumerate(subj):
        if rr > 500:
            outlier[0].append(rr) # add the rr int to our list
            outlier[1].append(idx2) # Add its index to our list
    subj = np.delete(subj, outlier[1]) # Remove all found outliers from our subject 
    rrs.append(subj) # Add it to our new rr interval list
    outlier_list.append(outlier)

23it [00:00, 44.48it/s]


In [5]:
 # Setup subset dictionary
subset_list = {}
for x in record_list:
    subset_list[x] = []

In [6]:
subset_len_sec = 25 # Set the time we are going to subset by
subset_len_samp = subset_len_sec*250 # Get that timme in samplse

for idx, subj in tqdm(enumerate(rrs)):
    samp = 0
    while samp < len(subj):
        subs_len = 0
        subs = []
        while subs_len < subset_len_samp and samp<len(subj):
            rr = subj[samp]
            subs.append(rr)
            subs_len+=rr
            samp+=1
        subset_list[record_list[idx]].append(subs)

23it [00:01, 18.99it/s]


In [7]:
subset_record_list = []
reload_flag=True
for idx, x in enumerate(tqdm(record_list)): 
    subset_record_list = []
    for num, subset in enumerate(subset_list[x]):
        if not os.path.exists('mit_bih_subset/'+x+ '-'+str(num)+'.csv') or reload_flag:
            np.savetxt('mit_bih_subset/'+x+ '-'+str(num)+'.csv', subset, delimiter=",",  fmt='%s') 
            
            subset_record_list.append(x+ '-'+str(num)+'.csv')
    if not os.path.exists('mit_bih_subset/'+x+ '-'+str(num)+'.csv') or reload_flag:
        np.savetxt('mit_bih_subset/'+x+'_subsetlist.csv', subset, delimiter=",",  fmt='%s') 
            # We'll load the complete list of subjects as well so that we can easily recreate the file names

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [02:34<00:00,  6.71s/it]
